In [1]:
# !pip install sentence_transformers

In [2]:
import pickle
from tqdm import tqdm
from pprint import pprint
from google.colab import drive

from sentence_transformers import SentenceTransformer, util

In [9]:
drive.mount('/content/drive')
path = 'drive/MyDrive/ryan/'

with open(path + 'music_categories.pkl', 'rb') as f:
  music_categories = pickle.load(f)

with open(path + 'other_categories.pkl', 'rb') as f:
  other_categories = pickle.load(f)

with open(path + 'music_categories_in_table.pkl', 'rb') as f:
  music_categories_in_table = pickle.load(f)

with open(path + 'all_categories_in_table.pkl', 'rb') as f:
  all_categories_in_table = pickle.load(f)
music_categories[0][1] = 'Light classical music'
music_categories[1][1] = 'Heavy classic rock'
music_categories[3][1] = 'Pop music'
music_categories[4][1] = 'Folk music'
music_categories[5][1] = 'Listening songs'
music_categories[18][1] = 'Alternative music'
music_categories[15][1] = 'Soul music'
other_categories[5][1] = 'How to do / education'

music_categories.extend(other_categories)
all_categories_in_data = music_categories
all_categories_in_data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[['Preference music-Yes [Light classic]', 'Light classical music'],
 ['Preference music-Yes [Heavy classic]', 'Heavy classic rock'],
 ['Preference music-Yes [Opera]', 'Opera'],
 ['Preference music-Yes [Dutch pop music]', 'Pop music'],
 ['Preference music-Yes [Dutch folk music]', 'Folk music'],
 ['Preference music-Yes [Dutch listening songs]', 'Listening songs'],
 ['Preference music-Yes [Rock]', 'Rock'],
 ['Preference music-Yes [Pop classics]', 'Pop classics'],
 ['Preference music-Yes [Top 40 pop]', 'Top 40 pop'],
 ['Preference music-Yes [Soft pop]', 'Soft pop'],
 ['Preference music-Yes [Hiphop/rap]', 'Hiphop/rap'],
 ['Preference music-Yes [World music (Latin, Arabic, African, etc.)]',
  'World music (Latin, Arabic, African, etc.)'],
 ['Preference music-Yes [Country/folk music]', 'Country/folk music'],
 ['Preference music-Yes [Jazz]', 'Jazz'],
 ['Preference music-Yes [Blues]', 'Blues'],
 ['Preference music-Yes [Soul]', 'Soul music'],
 ['Preference music-Yes [R&B]', 'R&B'],
 ['Preference

In [4]:
def get_score_mapping(categories, categories_table):
  model = SentenceTransformer('all-mpnet-base-v2')
  data_score = {}
  for category_data_actual, category_data in tqdm(categories):
    data_score[category_data_actual] = {
        'id': -1,
        'data': category_data_actual,
        'table': '',
        'score': 0
    }
    embedding_data = model.encode(category_data, convert_to_tensor = True)
    
    for id, category_table in categories_table:
      embedding_table = model.encode(category_table, convert_to_tensor = True)
      cosine_score = round((util.cos_sim(embedding_data, embedding_table)).item(), 2)
      
      if (cosine_score > data_score[category_data_actual]['score']):
        data_score[category_data_actual]['id'] = id
        data_score[category_data_actual]['score'] = cosine_score
        data_score[category_data_actual]['table'] = category_table
  return data_score

In [10]:
mappings = [
    [music_categories, all_categories_in_table, 'music_all_mapping.pkl'],
    [music_categories, music_categories_in_table, 'music_music_mapping.pkl'],
    [other_categories, all_categories_in_table, 'other_all_mapping.pkl'],
    [all_categories_in_data, all_categories_in_table, 'all_all_mapping.pkl']
]
    
for categories_data, categories_table, pkl_filename in mappings:
    scores = get_score_mapping(categories_data, categories_table )  
    with open(pkl_filename, 'wb') as file:
        pickle.dump(scores, file)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

100%|██████████| 36/36 [49:15<00:00, 82.10s/it]
